In [ ]:
import numpy as np
import math
import importlib
import sys
import time
import gc
from joblib import Parallel, delayed, parallel_backend

In [ ]:
gc.collect(2)

In [ ]:
# just setting up logging

import logging
from logging import debug

def config_log():
    FORMAT = "%(message)s"
    LEVEL = logging.ERROR
    STREAM = sys.stdout
    importlib.reload(logging)
    logging.basicConfig(format=FORMAT, level=LEVEL, stream=STREAM)
    
config_log()

# Explicit Triangle Counting

In [ ]:
def ExactTriangles(A, gamma):
    A3 = A.dot(A.dot(A))
    return math.floor(np.trace(A3) / 6)

# Algorithm 1

In [ ]:
# args: adjacency matrix of size n

def TraceTriangleN(A, gamma, parallel = True):
    # line 1
    
    n = A.shape[0]
    debug("n: {0}".format(n))
    
    # line 2
    
    M = math.ceil(gamma * math.pow(math.log(n), 2))
    debug("M: {0}".format(M))
    
    # line 3
    
    if parallel:
        with parallel_backend('threading', n_jobs=8):
            T = Parallel()(delayed(TraceTriangleN_Loop)(A, n) for i in range(M))
    else:
        T = [TraceTriangleN_Loop(A, n) for i in range(M)]
    # line 7
    
    # line 8
    
    Delta = 0
    
    for i in range(M):
        Delta += T[i] / M
        
    return Delta

def TraceTriangleN_Loop(A, n):
    #line 4

    x = np.random.normal(size = n)

    # line 5

    y = A.dot(x)

    # line 6

    return y.T.dot(A.dot(y)) / 6


# Modified  Algorithm 1

In [ ]:
# args: adjacency matrix of size n

def TraceTriangleR(A, gamma, parallel=True):
    # line 1
    
    n = A.shape[0]
    debug("n: {0}".format(n))
    
    # line 2
    
    M = math.ceil(gamma * math.pow(math.log(n), 2))
    debug("M: {0}".format(M))
    
    # line 3
    
    if parallel:
        with parallel_backend('threading', n_jobs=8):
            T = Parallel()(delayed(TraceTriangleR_Loop)(A, n) for i in range(M))
    else:
        T = [TraceTriangleR_Loop(A, n) for i in range(M)]
        
    # line 7
    
    # line 8
    
    Delta = 0
    
    for i in range(M):
        Delta += T[i] / M
        
    return Delta

def TraceTriangleR_Loop(A, n):
    #line 4

    x = np.random.choice(np.array([-1, 1]), size=n)

    debug(x)

    # line 5

    y = A.dot(x)

    # line 6

    return y.T.dot(A.dot(y)) / 6

# Algorithm 2

In [ ]:
def TraceTriangleM(A, gamma):
    # line 1
    
    n = A.shape[0]
    
    # line 2
    
    D_vec = np.random.choice(np.array([-1, 1]), size=n)
    
    D = np.diagflat(D_vec)
    
    debug(D)
    
    # line 3
    
    M = math.ceil(gamma * math.pow(math.log(n), 2))
    debug("M: {0}".format(M))
    
    # line 4
    
    T = np.zeros(M)
    AD = A.dot(D)
    
    for i in range(M):
        T[i] = TraceTriangleM_Loop(A, AD, n)
        
    # line 9
    
    Delta = 0
    
    for i in range(M):
        Delta += T[i] / M
        
    return Delta

def TraceTriangleM_Loop(A, AD, n):
    # line 5
        
    j = np.random.randint(low = 1, high = n+1)
    debug(j)

    # line 6

    if j == 1:
        x = np.full(n, math.sqrt(1 / n))
    else:
        x = np.zeros(n)

        for k in range(n):
            x[k] = math.sqrt(2 / n) * math.cos(math.pi * (k + 1/2) * j / n)

    # line 7

    y = AD.dot(x)

    # line 8

    return (n * y.T.dot(A.dot(y)) / 6)

# Evaluation

Each algorithm is evaluated against several datasets

In [ ]:
def do_test(method, matrix, rounds, correct=None, gamma=None):
    results = []
    times = []
    
    for _ in range(rounds):
        start = time.time()
        ans = method(matrix, gamma)
        delta = time.time() - start
        times.append(delta)
        results.append(ans)
    
    mean_result = sum(results) / len(results)
    mean_time = sum(times) / len(times)
    
    print(method.__name__)
    print("Time: {0}".format(mean_time))
    print("Answer: {0}".format(mean_result))
    

def test_with(matrix, gamma, rounds):
    do_test(ExactTriangles, matrix, rounds, gamma=1)
    do_test(TraceTriangleN, matrix, rounds, gamma=1)
    do_test(TraceTriangleR, matrix, rounds, gamma=1)

## 1 triangle

In [ ]:
OneTri = np.zeros((3, 3))
i,j = np.indices(OneTri.shape)

OneTri[i==j-2] = 1
OneTri[i==j-1] = 1
OneTri[i==j+1] = 1
OneTri[i==j+2] = 1

## 10x10 penta-diagonal

In [ ]:
A10 = np.zeros((10,10))
i,j = np.indices(A10.shape)

A10[i==j-2] = 1
A10[i==j-1] = 1
A10[i==j+1] = 1
A10[i==j+2] = 1

## 1000x1000 penta-diagonal

In [ ]:
A1k = np.zeros((1000,1000))
i,j = np.indices(A1k.shape)

A1k[i==j-2] = 1
A1k[i==j-1] = 1
A1k[i==j+1] = 1
A1k[i==j+2] = 1

## 10000x10000 penta-diagonal

In [ ]:
A10k = np.zeros((10000,10000))
i,j = np.indices(A10k.shape)

A10k[i==j-2] = 1
A10k[i==j-1] = 1
A10k[i==j+1] = 1
A10k[i==j+2] = 1

In [ ]:
test_with(A10k, gamma=1, rounds=1)